In [1]:
# MLB Attendance Data - DATA 301 Final Project

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import time

In [3]:
team_names = {"ARI": ["ARI"],
              "ATL": ["ATL", "MLN", "BSN"], 
              "BAL": ["BAL", "SLB", "MLA"], 
              "BOS": ["BOS"], 
              "CHC": ["CHC"], 
              "CHW": ["CHW"], 
              "CIN": ["CIN"], 
              "CLE": ["CLE"], 
              "COL": ["COL"], 
              "DET": ["DET"], 
              "HOU": ["HOU"], 
              "KCR": ["KCR"], 
              "LAA": ["LAA", "ANA", "CAL"], 
              "LAD": ["LAD", "BRO"], 
              "MIA": ["MIA", "FLA"], 
              "MIL": ["MIL", "SEP"],
              "MIN": ["MIN", "WSH"], 
              "NYM": ["NYM"],
              "NYY": ["NYY"],
              "OAK": ["OAK", "KCA", "PHA"],
              "PHI": ["PHI"],
              "PIT": ["PIT"],
              "SDP": ["SDP"],
              "SFG": ["SFG", "NYG"], 
              "SEA": ["SEA"], 
              "STL": ["STL"], 
              "TBD": ["TBD", "TBR"],
              "TEX": ["TEX", "WSA"], 
              "TOR": ["TOR"],
              "WSN": ["WSN", "MON"]
              }

teams = dict.fromkeys(team_names)

In [11]:
for key in team_names.keys():
    games = []
    print(key)
    for year in range(1876,2016):
        for team in team_names[key]:
            url = "http://www.baseball-reference.com/teams/" + team + "/" + str(year) + "-schedule-scores.shtml"
            page = requests.get(url)
            season = BeautifulSoup(page.text, "html.parser")

            table = season.find("div", {"id": "div_team_schedule"},class_="table_container")
            if table is not None:
                table = table.find_all("tr", class_="")
                for row in table[1:-1]:
                    if "thead" not in row["class"]:
                        values = row.find_all("td")
                        games.append({
                                "Year": year,
                                "DOW" : values[2].text.split(" ")[0][0:-1],
                                "Month" : values[2].text.split(" ")[1],
                                "Opp" : values[6].text,
                                "W/L" : values[7].text[0],
                                "Game" : values[1].text,
                                "WO" : "Y" if len(values[7].text) > 1 else "N",
                                "GB" : 0 if "Tied" in values[13].text 
                                        else (values[13].text).replace(" ", "").replace("up", "-"),
                                "Time" : values[18].text,
                                "Attendance" : values[19].text.replace(",", ""),
                                "Streak" : len(values[20].text),
                                "Home" : values[5].text,
                                "Pct" : 0 if (float(values[11].text.split("-")[0]) + float(values[11].text.split("-")[1])) == 0 
                                            else float(values[11].text.split("-")[0]) / (float(values[11].text.split("-")[0]) + float(values[11].text.split("-")[1])),
                                "RS" : int(values[8].text),
                                "RA" : int(values[9].text),
                                "Pitcher" : (values[14].find("a")["title"] if values[14].find("a") is not None else "") 
                                            if values[7].text[0] == "W" else (values[15].find("a")["title"] if values[15].find("a") is not None else ""),
                                "OPitcher" : (values[15].find("a")["title"] if values[15].find("a") is not None else "") 
                                            if values[7].text[0] == "W" else (values[14].find("a")["title"] if values[14].find("a") is not None else ""),
                                "Team" : team,
                            })
                break

            time.sleep(0.1)
    d = pd.DataFrame(games)
    if "Attendance" in d:
        d["Attendance"] = pd.to_numeric(d["Attendance"])
    teams[team] = d

CIN
ATL
NYY
LAD
ARI
CHC
PHI
BOS
LAA
TEX
WSN
HOU
PIT
TOR
CHW
TBD
BAL
KCR
SDP
OAK
COL
MIL
CLE
MIA
DET
STL
SEA
MIN
SFG
NYM


In [21]:
copy = teams.copy()

In [23]:
teams["NYY"]

,Attendance,DOW,GB,Game,Home,Month,OPitcher,Opp,Pct,Pitcher,RA,RS,Streak,Team,Time,W/L,WO,Year
0,NaN,Wednesday,1.0,1,@,Apr,,WSH,0.000000,,3,1,1,NYY,,L,N,1903
1,NaN,Thursday,1.0,2,@,Apr,,WSH,0.500000,,2,7,1,NYY,,W,N,1903
2,NaN,Friday,1.5,3,@,Apr,,WSH,0.333333,,7,1,1,NYY,,L,N,1903
3,NaN,Saturday,1.5,4,@,Apr,,WSH,0.500000,,1,11,1,NYY,,W,N,1903
4,NaN,Monday,2.5,5,@,Apr,,PHA,0.400000,,6,0,1,NYY,,L,N,1903
5,NaN,Tuesday,2.5,6,@,Apr,,PHA,0.333333,,7,3,2,NYY,,L,N,1903
6,NaN,Wednesday,2.5,7,@,Apr,,PHA,0.428571,,4,5,1,NYY,,W,N,1903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17651,39476.0,Monday,5.0,156,,Sep,Eduardo Rodriguez,BOS,0.551282,Ivan Nova,5,1,1,NYY,N,L,N,2015
17652,38512.0,Tuesday,5.5,157,,Sep,Rick Porcello,BOS,0.547771,Michael Pineda,10,4,2,NYY,N,L,N,2015


In [14]:
#df.to_csv(file_name, sep='\t')

import json

# load from file:
with open('/path/Final\ Project/MLB-attendence-data/save.json', 'r') as f:
    try:
        data = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}

# save to file:
with open('/path/to/my_file.json', 'w') as f:
    data['new_key'] = [1, 2, 3]
    json.dump(data, f)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/my_file.json'